In [4]:
import pandas as pd
import numpy as np
from dateutil import parser
import re
from fuzzywuzzy import process, fuzz
from sklearn.model_selection import train_test_split
from datetime import datetime
import json

In [5]:
TRAIN_DATA_ADDRESS = "datathon-dataset/train.csv"
train_data = pd.read_csv(TRAIN_DATA_ADDRESS)
train_data

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3335828751.py:2: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(TRAIN_DATA_ADDRESS)


,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

In [7]:
def modify_column_names(col):
    col = col.strip()
    col = col.lower()
    col = col.replace(" ", "_")
    return col

In [8]:
train_data.columns = [modify_column_names(col) for col in train_data.columns]

In [9]:
columns_to_drop = [
    "lise_adi_diger",
    "lise_turu",
    "lise_bolum_diger",
    "hangi_stk'nin_uyesisiniz?",
    "daha_once_baska_bir_universiteden_mezun_olmus",
    "daha_önceden_mezun_olunduysa,_mezun_olunan_üniversite",
    "uye_oldugunuz_kulubun_ismi",
    "spor_dalindaki_rolunuz_nedir?",
    "girisimcilikle_ilgili_deneyiminiz_var_mi?",
    "baska_bir_kurumdan_burs_aliyor_mu?",
    "burs_aldigi_baska_kurum",
    "baska_kurumdan_aldigi_burs_miktari",
    "lise_bolumu",
    "girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?",
]

In [10]:
train_data.drop(columns=columns_to_drop, inplace=True)

In [11]:
for column in train_data.columns:
    if train_data[column].dtype.name == "object":
        train_data[column] = train_data[column].astype("str")

In [12]:
train_data.describe(include="all")

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,...,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id
count,65125.000000,65124.000000,65125,65125,65125,65125,65125,65125,22440.000000,65125,...,65125,65125,65125,65125,65125,65125,65125,65125,65125,65125.000000
unique,NaN,NaN,5,13499,2231,320,528,5,NaN,5,...,4,9,29,3,3,3,3,3,5,NaN
top,NaN,NaN,Kadın,1/1/70 2:00,İstanbul,İstanbul,İSTANBUL ÜNİVERSİTESİ,Devlet,NaN,Hayır,...,Evet,nan,1.0,Hayır,Hayır,Hayır,nan,Evet,nan,NaN
freq,NaN,NaN,32077,1655,8682,16625,2083,37885,NaN,51978,...,28559,20378,16186,33710,42243,37716,28308,48872,36533,NaN
mean,2018.277298,32.086466,NaN,NaN,NaN,NaN,NaN,NaN,34.226181,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32562.000000
std,2.669979,18.139239,NaN,NaN,NaN,NaN,NaN,NaN,41.632412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18800.112478
min,2014.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,2016.000000,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16281.000000
50%,2019.000000,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32562.000000
75%,2021.000000,44.000000,NaN,NaN,NaN,NaN,NaN,NaN,75.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48843.000000


In [13]:
train_data.columns

Index(['basvuru_yili', 'degerlendirme_puani', 'cinsiyet', 'dogum_tarihi',
       'dogum_yeri', 'ikametgah_sehri', 'universite_adi', 'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu?', 'bölüm',
       'universite_kacinci_sinif', 'universite_not_ortalamasi', 'lise_adi',
       'lise_sehir', 'lise_mezuniyet_notu', 'anne_egitim_durumu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id'],
      dtype='object')

In [14]:
if "degerlendirme_puani" in train_data.columns:
    train_data["degerlendirme_puani"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/1054753668.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["degerlendirme_puani"].fillna(0, inplace=True)


In [15]:
cinsiyet_map = {
    "Erkek": 0,
    "Kadın": 1,
    "Belirtmek istemiyorum": 2,
    "ERKEK": 0,
    "nan": 3,
}

In [16]:
train_data["cinsiyet"] = train_data["cinsiyet"].map(cinsiyet_map)

In [17]:
train_data["cinsiyet"].value_counts(dropna=False)

cinsiyet
0    32764
1    32077
3      169
2      115
Name: count, dtype: int64

In [18]:
train_data["dogum_tarihi"].value_counts()

dogum_tarihi
1/1/70 2:00         1655
1/1/00 0:00          179
nan                  177
1/1/01 0:00          154
1/1/1997             139
                    ... 
29/08/2000             1
7/30/1991              1
14/05/1996             1
15/12/1997             1
12.11.2003 00:00       1
Name: count, Length: 13499, dtype: int64

In [19]:
def parse_date(date_string):
    try:
        result = parser.parse(date_string, dayfirst=True)
        if (
            result.year > datetime.now().year
            or result.year <= datetime.now().year - 100
        ):
            raise ValueError()
        else:
            return result
    except ValueError:
        return pd.NA

In [20]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,...,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id
0,2014,52.0,0,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Emekli,0,1,Evet,Evet,Hayır,Evet,nan,nan,0
1,2014,30.0,0,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,Emekli,0,1,Hayır,Hayır,Hayır,Evet,nan,nan,1
2,2014,18.0,0,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,Evet,Özel Sektör,6,Evet,Hayır,Hayır,Hayır,nan,nan,2
3,2014,40.0,0,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,Emekli,0,10,Evet,Hayır,Hayır,Evet,nan,nan,3
4,2014,24.0,0,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,Emekli,0,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,Evet,Özel Sektör,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120
65121,2022,0.0,0,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,Evet,Özel Sektör,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121
65122,2022,0.0,0,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,Hayır,-,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122
65123,2022,0.0,0,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,Evet,Kamu,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123


In [21]:
train_data["dogum_tarihi"] = train_data["dogum_tarihi"].apply(parse_date)

In [22]:
train_data["dogum_tarihi"]

0        1994-06-04 00:00:00
1        1993-11-06 00:00:00
2        1986-01-15 00:00:00
3        1991-04-06 00:00:00
4                       <NA>
                ...         
65120    2003-06-18 00:00:00
65121    2002-04-18 00:00:00
65122    1999-10-23 00:00:00
65123    2000-11-17 00:00:00
65124    2000-04-27 00:00:00
Name: dogum_tarihi, Length: 65125, dtype: object

In [23]:
train_data["dogum_tarihi"] = pd.to_datetime(
    train_data["dogum_tarihi"], format="%Y-%m-%d %H:%M:%S"
)

In [24]:
train_data["dogum_tarihi"].isna().sum()

4780

In [25]:
train_data["dogum_yili"] = train_data["dogum_tarihi"].dt.year
train_data["dogum_ayi"] = train_data["dogum_tarihi"].dt.month
train_data.drop(columns="dogum_tarihi", inplace=True)

In [26]:
train_data["dogum_yili"].fillna(train_data["dogum_yili"].mode()[0], inplace=True)
train_data["dogum_ayi"].fillna(train_data["dogum_ayi"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3503408563.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["dogum_yili"].fillna(train_data["dogum_yili"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3503408563.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [27]:
train_data["dogum_yeri"].unique()

array(['Altindag, Ankara', 'Üsküdar', 'Samsun', ..., 'bergama/ izmir',
       'Tacikistan', 'Ankara keçiören'], dtype=object)

In [28]:
train_data["dogum_yeri"].value_counts()

dogum_yeri
İstanbul             8682
Ankara               4045
İzmir                2170
Istanbul             1894
Adana                1810
                     ... 
körfez/kocaeli          1
GEDIZ                   1
İstanbul Bakırköy       1
İstanbul-Şişli          1
Ankara keçiören         1
Name: count, Length: 2231, dtype: int64

In [29]:
turkish_cities = {}

with open("datathon-dataset/processed-data/turkish_cities.json") as json_file:
    data = json.load(json_file)
    turkish_cities = {int(key): value for key, value in data.items()}
    print(turkish_cities)

{1: 'Adana', 2: 'Adıyaman', 3: 'Afyonkarahisar', 4: 'Ağrı', 5: 'Amasya', 6: 'Ankara', 7: 'Antalya', 8: 'Artvin', 9: 'Aydın', 10: 'Balıkesir', 11: 'Bilecik', 12: 'Bingöl', 13: 'Bitlis', 14: 'Bolu', 15: 'Burdur', 16: 'Bursa', 17: 'Çanakkale', 18: 'Çankırı', 19: 'Çorum', 20: 'Denizli', 21: 'Diyarbakır', 22: 'Edirne', 23: 'Elazığ', 24: 'Erzincan', 25: 'Erzurum', 26: 'Eskişehir', 27: 'Gaziantep', 28: 'Giresun', 29: 'Gümüşhane', 30: 'Hakkâri', 31: 'Hatay', 32: 'Isparta', 33: 'Mersin', 34: 'İstanbul', 35: 'İzmir', 36: 'Kars', 37: 'Kastamonu', 38: 'Kayseri', 39: 'Kırklareli', 40: 'Kırşehir', 41: 'Kocaeli', 42: 'Konya', 43: 'Kütahya', 44: 'Malatya', 45: 'Manisa', 46: 'Kahramanmaraş', 47: 'Mardin', 48: 'Muğla', 49: 'Muş', 50: 'Nevşehir', 51: 'Niğde', 52: 'Ordu', 53: 'Rize', 54: 'Sakarya', 55: 'Samsun', 56: 'Siirt', 57: 'Sinop', 58: 'Sivas', 59: 'Tekirdağ', 60: 'Tokat', 61: 'Trabzon', 62: 'Tunceli', 63: 'Şanlıurfa', 64: 'Uşak', 65: 'Van', 66: 'Yozgat', 67: 'Zonguldak', 68: 'Aksaray', 69: 'Bayburt

In [30]:
def clean_city_name(city):
    if pd.isna(city) or not isinstance(city, str):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    city = city.lower().strip()

    # Remove any digits
    city = re.sub(r"\d+", "", city)

    # Remove any parentheses and their contents
    city = re.sub(r"\([^)]*\)", "", city)

    # Remove common words that might appear (customize this list as needed)
    remove_words = ["şehir", "ili", "ilçesi", "merkez"]
    for word in remove_words:
        city = city.replace(word, "")

    # Remove any remaining punctuation and extra whitespace
    city = re.sub(r"[^\w\s]", "", city)
    city = " ".join(city.split())

    # Use fuzzy matching to find the closest match
    city_list = list(turkish_cities.values())
    closest_match, score = process.extractOne(city, city_list)

    if score >= 80:
        for key, value in turkish_cities.items():
            if value == closest_match:
                return key
    else:
        return pd.NA

In [31]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [32]:
train_data["dogum_yeri"] = train_data["dogum_yeri"].apply(clean_city_name)

In [33]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,<NA>,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [34]:
train_data["dogum_yeri"].unique()

array([6, <NA>, 55, 21, 34, 24, 58, 38, 64, 53, 25, 46, 42, 35, 61, 36,
       18, 1, 54, 48, 31, 33, 44, 45, 19, 17, 7, 26, 74, 70, 65, 12, 23,
       28, 52, 10, 16, 43, 78, 11, 56, 2, 39, 9, 67, 41, 59, 60, 63, 37,
       22, 80, 30, 20, 4, 50, 71, 32, 68, 72, 27, 47, 14, 3, 51, 5, 81,
       66, 57, 77, 69, 49, 13, 62, 29, 15, 73, 76, 75, 8, 79, 40],
      dtype=object)

In [35]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 31 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   basvuru_yili                                          65125 non-null  int64  
 1   degerlendirme_puani                                   65125 non-null  float64
 2   cinsiyet                                              65125 non-null  int64  
 3   dogum_yeri                                            61102 non-null  object 
 4   ikametgah_sehri                                       65125 non-null  object 
 5   universite_adi                                        65125 non-null  object 
 6   universite_turu                                       65125 non-null  object 
 7   burslu_ise_burs_yuzdesi                               22440 non-null  float64
 8   burs_aliyor_mu?                                       65

In [36]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,<NA>,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [37]:
train_data["ikametgah_sehri"]

0          Ankara
1        İstanbul
2        İstanbul
3        İstanbul
4          Ankara
           ...   
65120    İstanbul
65121       Muğla
65122      Mersin
65123      Samsun
65124       Hatay
Name: ikametgah_sehri, Length: 65125, dtype: object

In [38]:
train_data["ikametgah_sehri"] = train_data["ikametgah_sehri"].apply(clean_city_name)

In [39]:
train_data["dogum_yeri"] = train_data.apply(
    lambda row: (
        row["ikametgah_sehri"]
        if pd.isnull(row["dogum_yeri"]) or row["dogum_yeri"] == ""
        else row["dogum_yeri"]
    ),
    axis=1,
)

In [40]:
train_data[train_data["dogum_yeri"].isna()]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
38,2014,26.0,1,<NA>,<NA>,HALİÇ ÜNİVERSİTESİ,Özel,100.0,Evet,Isletme Enformatigi,...,2,Hayır,Hayır,Hayır,Hayır,nan,nan,38,1995.0,12.0
69,2014,12.0,0,<NA>,<NA>,İSTANBUL AREL ÜNİVERSİTESİ,Özel,100.0,Evet,International Relations,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,69,1992.0,12.0
71,2014,48.0,1,<NA>,<NA>,FATİH ÜNİVERSİTESİ,Özel,100.0,Evet,Felsefe,...,3,Evet,Evet,Hayır,Evet,nan,nan,71,1992.0,4.0
81,2014,26.0,1,<NA>,<NA>,GİRNE AMERİKAN ÜNİVERSİTESİ,Özel,100.0,Evet,Halkla İlişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,81,1989.0,2.0
85,2014,56.0,1,<NA>,<NA>,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,İKTİSAT,...,1,Evet,Hayır,Hayır,Evet,nan,nan,85,1993.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62815,2022,23.0,1,<NA>,<NA>,Kütahya Dumlupınar Üniversitesi,Devlet,NaN,Hayır,Diğer,...,2.0,Evet,Hayır,Evet,nan,Hayır,nan,62815,2002.0,9.0
63428,2022,19.0,1,<NA>,<NA>,Dicle Üniversitesi,Devlet,NaN,Hayır,Psikoloji,...,4.0,Hayır,Hayır,Evet,nan,Hayır,nan,63428,2002.0,1.0
63429,2022,19.0,1,<NA>,<NA>,Çanakkale Onsekiz Mart Üniversitesi,Devlet,NaN,Hayır,Sosyoloji,...,1.0,Evet,Hayır,Hayır,nan,Evet,nan,63429,2002.0,2.0
63953,2022,15.0,1,<NA>,<NA>,Çanakkale Onsekiz Mart Üniversitesi,Devlet,NaN,Hayır,Matematik,...,2.0,Evet,Hayır,Hayır,nan,Hayır,nan,63953,1999.0,10.0


In [41]:
train_data["ikametgah_sehri"] = train_data.apply(
    lambda row: (
        row["dogum_yeri"]
        if pd.isnull(row["ikametgah_sehri"]) or row["ikametgah_sehri"] == ""
        else row["ikametgah_sehri"]
    ),
    axis=1,
)

In [42]:
train_data["dogum_yeri"].fillna(train_data["dogum_yeri"].mode()[0], inplace=True)
train_data["ikametgah_sehri"].fillna(
    train_data["ikametgah_sehri"].mode()[0], inplace=True
)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/4291801807.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["dogum_yeri"].fillna(train_data["dogum_yeri"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/4291801807.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [43]:
train_data[train_data["ikametgah_sehri"].isna()]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi


In [44]:
train_data[train_data["dogum_yeri"].isna()]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi


In [45]:
train_data["dogum_yeri"].value_counts(dropna=False)

dogum_yeri
34    13082
6      4594
35     2983
16     2035
1      1973
      ...  
75      110
69       82
62       58
40        3
79        1
Name: count, Length: 81, dtype: int64

In [46]:
train_data["dogum_yeri"].unique()

array([ 6, 34, 55, 21, 24, 58, 38, 64, 53, 25, 46, 42, 35, 61, 36, 18, 41,
        1, 54, 48, 31, 33, 44, 45,  7, 19, 17, 26, 74, 70, 65, 12, 23, 28,
       52, 10, 16, 43, 78, 11, 56,  2, 39,  9, 67, 59, 60, 63, 37, 22, 80,
       30, 20,  4, 72, 27, 50, 71, 32, 68, 47, 14,  3, 51,  5, 81, 66, 57,
       77, 69, 49, 13, 62, 29, 15, 73, 76, 75,  8, 79, 40])

In [47]:
train_data["ikametgah_sehri"].unique()

array([ 6, 34, 38, 35, 46, 42, 61, 36, 18, 41,  1, 52, 80,  7, 26, 74, 12,
       33, 16, 45, 55, 78, 21,  2, 39, 54,  9, 48, 59, 67, 25, 30, 20, 72,
       27, 10, 50, 32, 63, 70, 24, 44, 51, 53, 68, 43,  3, 64, 66, 14, 23,
       37, 81, 57, 71,  4, 28, 31, 17, 56, 58, 73, 47, 65, 69, 19, 13,  5,
       62, 22, 75, 60, 77, 15, 29,  8, 76, 49, 11, 79, 40])

In [48]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [49]:
train_data["universite_adi"].unique()

array(['İHSAN DOĞRAMACI BİLKENT', 'ULUSLARARASI KIBRIS ÜNİVERSİTESİ',
       'İSTANBUL ŞEHİR ÜNİVERSİTESİ', 'TURGUT ÖZAL ÜNİVERSİTESİ',
       'İSTANBUL TİCARET ÜNİVERSİTESİ', 'İSTANBUL MEDİPOL ÜNİVERSİTESİ',
       'FATİH SULTAN MEHMET VAKIF', 'BAHÇEŞEHİR ÜNİVERSİTESİ',
       'NUH NACİ YAZGAN ÜNİVERSİTESİ', 'FATİH ÜNİVERSİTESİ',
       'BAŞKENT ÜNİVERSİTESİ', 'ÇAĞ ÜNİVERSİTESİ', 'İZMİR ÜNİVERSİTESİ',
       'ZİRVE ÜNİVERSİTESİ', 'YAŞAR ÜNİVERSİTESİ',
       'İSTANBUL AREL ÜNİVERSİTESİ', 'TED ÜNİVERSİTESİ',
       'YENİ YÜZYIL ÜNİVERSİTESİ', 'İZMİR EKONOMİ ÜNİVERSİTESİ',
       'İSTANBUL BİLGİ ÜNİVERSİTESİ', 'İSTANBUL GELİŞİM ÜNİVERSİTESİ',
       'BEYKENT ÜNİVERSİTESİ', 'AVRASYA ÜNİVERSİTESİ',
       'OKAN ÜNİVERSİTESİ', 'KOÇ ÜNİVERSİTESİ', 'HALİÇ ÜNİVERSİTESİ',
       'IŞIK ÜNİVERSİTESİ', 'ATILIM ÜNİVERSİTESİ',
       'NİŞANTAŞI ÜNİVERSİTESİ', 'İSTANBUL AYDIN ÜNİVERSİTESİ',
       'UFUK ÜNİVERSİTESİ', 'SABANCI ÜNİVERSİTESİ',
       'İSTANBUL 29 MAYIS ÜNİVERSİTESİ', 'MALTEPE ÜNİVERSİ

In [50]:
universities = pd.read_html(
    "https://tr.wikipedia.org/wiki/T%C3%BCrkiye%27deki_%C3%BCniversiteler_listesi"
)[0]["Üniversite"].to_dict()

In [51]:
def clean_university_name(university):
    if pd.isna(university):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    university = university.lower().strip()

    # Remove any digits
    university = re.sub(r"\d+", "", university)

    # Remove any parentheses and their contents
    university = re.sub(r"\([^)]*\)", "", university)

    # Remove any remaining punctuation and extra whitespace
    university = re.sub(r"[^\w\s]", "", university)
    university = " ".join(university.split())

    # Use fuzzy matching to find the closest match
    university_list = list(universities.values())
    closest_match, score = process.extractOne(university, university_list)

    if score >= 80:
        for key, value in universities.items():
            if value == closest_match:
                return key
    else:
        return university

In [52]:
train_data["universite_adi"] = train_data["universite_adi"].apply(clean_university_name)

In [53]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23,Özel,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112,Özel,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107,Özel,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176,Devlet,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76,Devlet,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146,Devlet,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [54]:
train_data["universite_adi"].unique()

array([23, 0, 112, 169, 122, 116, 99, 93, 153, 59, 21, 174, 188, 143, 105,
       26, 141, 109, 113, 108, 198, 118, 127, 101, 102, 20, 107, 28, 132,
       104, 128, 27, 165, 22, 115, 134, 68, 133, 126, 110,
       'istanbul kavram meslek', 13, 200, 130, 175, 131, 31, 50, 70, 98,
       'beykoz lojistik meslek', 29, 14, 120, 78, 182, 15, 7, 159, 91,
       129, 'plato meslek yüksekokulu', 63, 47, 82,
       'alanya hamdullah emin paşa', 95, 183, 1, 39, 81,
       'istanbul kavram meslek yüksekokulu', 117, 124, 172, 96, 148, 5, 9,
       84, 52, 56, 184, 76, 155, 43, 186, 204, 195, 196, 'diğer', 53, 146,
       48, 87, 58, 162, 189, 149, 90, 61, 38, 138, 170, 176, 151, 66, 74,
       64, 45, 6, 136, 164, 71, 55, 150, 163, 79, 44, 171, 190, 80, 2,
       144, 135, 202, 192, 75, 177, 49, 147, 'nan', 193, 178, 203,
       'kapadokya meslek yüksekokulu', 54, 86, 168, 46, 37, 3, 57, 41,
       154, 60, 201, 194, 92, 137, 100, 89, 125, 97, 119, 94, 103, 179,
       140, 180, 114, 65, 160, 156

In [55]:
train_data["universite_adi"] = pd.to_numeric(
    train_data["universite_adi"],
    errors="coerce",
)
train_data["universite_adi"].fillna(
    train_data["universite_adi"].mode()[0], inplace=True
)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/706566557.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["universite_adi"].fillna(


In [56]:
train_data["universite_adi"].value_counts(dropna=False)

universite_adi
82.0     3369
84.0     2084
81.0     1954
90.0     1916
0.0      1820
         ... 
106.0       5
10.0        3
17.0        2
8.0         1
85.0        1
Name: count, Length: 200, dtype: int64

In [57]:
train_data["universite_adi"].unique()

array([ 23.,   0., 112., 169., 122., 116.,  99.,  93., 153.,  59.,  21.,
       174., 188., 143., 105.,  26., 141., 109., 113., 108., 198., 118.,
       127., 101., 102.,  20., 107.,  28., 132., 104., 128.,  27., 165.,
        22., 115., 134.,  68., 133., 126., 110.,  82.,  13., 200., 130.,
       175., 131.,  31.,  50.,  70.,  98.,  29.,  14., 120.,  78., 182.,
        15.,   7., 159.,  91., 129.,  63.,  47.,  95., 183.,   1.,  39.,
        81., 117., 124., 172.,  96., 148.,   5.,   9.,  84.,  52.,  56.,
       184.,  76., 155.,  43., 186., 204., 195., 196.,  53., 146.,  48.,
        87.,  58., 162., 189., 149.,  90.,  61.,  38., 138., 170., 176.,
       151.,  66.,  74.,  64.,  45.,   6., 136., 164.,  71.,  55., 150.,
       163.,  79.,  44., 171., 190.,  80.,   2., 144., 135., 202., 192.,
        75., 177.,  49., 147., 193., 178., 203.,  54.,  86., 168.,  46.,
        37.,   3.,  57.,  41., 154.,  60., 201., 194.,  92., 137., 100.,
        89., 125.,  97., 119.,  94., 103., 179., 14

In [58]:
train_data["universite_adi"].isna().sum()

0

In [59]:
train_data["universite_turu"].value_counts()

universite_turu
Devlet    37885
DEVLET    15377
Özel       8336
ÖZEL       3272
nan         255
Name: count, dtype: int64

In [60]:
university_type_map = {"Devlet": 0, "DEVLET": 0, "Özel": 1, "ÖZEL": 1, "nan": -1}

In [61]:
train_data["universite_turu"] = train_data["universite_turu"].map(university_type_map)

In [62]:
train_data["universite_turu"].unique()

array([ 1,  0, -1])

In [63]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100.0,Evet,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,100.0,Hayır,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,100.0,Hayır,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100.0,Evet,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,NaN,Evet,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,NaN,Hayır,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,NaN,Hayır,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,NaN,Hayır,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [64]:
train_data["universite_turu"].value_counts(dropna=False)

universite_turu
 0    53262
 1    11608
-1      255
Name: count, dtype: int64

In [65]:
train_data["burs_aliyor_mu?"].unique()

array(['Evet', 'Hayır', 'hayır', 'evet', 'EVET'], dtype=object)

In [66]:
is_there_scholarship = {"Evet": 1, "EVET": 1, "Hayır": 0, "hayır": 0}
train_data["burs_aliyor_mu?"] = train_data["burs_aliyor_mu?"].map(is_there_scholarship)

In [67]:
train_data["burs_aliyor_mu?"] = train_data["burs_aliyor_mu?"].fillna(0)

In [68]:
train_data["burs_aliyor_mu?"].value_counts(dropna=False)

burs_aliyor_mu?
0.0    52361
1.0    12764
Name: count, dtype: int64

In [69]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100.0,1.0,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,100.0,0.0,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,100.0,0.0,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100.0,1.0,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100.0,1.0,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,NaN,1.0,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,NaN,0.0,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,NaN,0.0,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,NaN,0.0,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [70]:
train_data["burslu_ise_burs_yuzdesi"].unique()

array([100. ,  99. ,  98. ,  91.5,  90. ,  88. ,  87.5,  85. ,  82.5,
        80. ,  75. ,  74. ,  72. ,  70. ,  69. ,  67.5,  67. ,  65. ,
        64. ,  63. ,  62.5,  62. ,  60. ,  57.5,  57. ,  55. ,  50. ,
        49. ,  48. ,  46. ,  45. ,  41.5,  41. ,  40. ,  37. ,  35. ,
        32. ,  30. ,  25. ,  20. ,  11. ,  10. ,   8. ,   7. ,   6. ,
         5. ,   4. ,   3. ,   2. ,   1. ,   0. ,   nan])

In [71]:
train_data["burslu_ise_burs_yuzdesi"] = train_data.apply(
    lambda row: 100 if row["burs_aliyor_mu?"] == 1 else 0, axis=1
)

In [72]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [73]:
train_data["burslu_ise_burs_yuzdesi"].value_counts(dropna=False)

burslu_ise_burs_yuzdesi
0      52361
100    12764
Name: count, dtype: int64

In [74]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,Bilgisayar Muhendisligi,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,Elektrik Elektronik Mühendisliği,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,Finans Yönetimi ve Pazarlama,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,Bilgisayar Mühendisliği,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,Siyaset Bilimi ve Uluslararası ilişkiler,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,Muhasebe,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,Turizm İşletmeciliği,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,Otomotiv Mühendisliği,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,Sanat Tarihi,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [75]:
train_data.columns

Index(['basvuru_yili', 'degerlendirme_puani', 'cinsiyet', 'dogum_yeri',
       'ikametgah_sehri', 'universite_adi', 'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu?', 'bölüm',
       'universite_kacinci_sinif', 'universite_not_ortalamasi', 'lise_adi',
       'lise_sehir', 'lise_mezuniyet_notu', 'anne_egitim_durumu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id',
       'dogum_yili', 'dogum_ayi'],
      dtype='object')

In [76]:
train_data["bölüm"].value_counts()

bölüm
Endüstri Mühendisliği                                     2434
İşletme                                                   2318
Bilgisayar Mühendisliği                                   1917
Hukuk                                                     1769
Makine Mühendisliği                                       1283
                                                          ... 
BEDEN EĞİTİMİ VE SPOR YÜKSEK OKULU/ REKREASYON               1
Beden Eğitimi ve Spor Yüksek Okulu Antrenörlük Eğitimi       1
özel eğitim zeö                                              1
müzik bölümü                                                 1
Nanoteknoloji Mühendisliği                                   1
Name: count, Length: 7749, dtype: int64

In [77]:
bolumler = pd.read_html("https://hangiuniversite.com/bolumler")[0]
bolumler = pd.DataFrame(bolumler)
majors = bolumler.apply(lambda x: x.str.lstrip("» ")).to_dict()["Bölüm"]

In [78]:
majors

{0: 'ACİL YARDIM VE AFET YÖNETİMİ(FAKÜLTE)',
 1: 'ACİL YARDIM VE AFET YÖNETİMİ(YÜKSEKOKUL)',
 2: 'ADLİ BİLİMLER',
 3: 'ADLİ BİLİŞİM MÜHENDİSLİĞİ',
 4: 'ADLİ BİLİŞİM MÜHENDİSLİĞİ(M.T.O.K.)',
 5: 'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ',
 6: 'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ(M.T.O.K.)',
 7: 'AİLE VE TÜKETİCİ BİLİMLERİ',
 8: 'AKTÜERYA BİLİMLERİ(FAKÜLTE)',
 9: 'ALMAN DİLİ VE EDEBİYATI',
 10: 'ALMANCA MÜTERCİM VE TERCÜMANLIK(FAKÜLTE)',
 11: 'ALMANCA MÜTERCİM VE TERCÜMANLIK(YÜKSEKOKUL)',
 12: 'ALMANCA ÖĞRETMENLİĞİ',
 13: 'AMERİKAN KÜLTÜRÜ VE EDEBİYATI',
 14: 'ANTRENÖRLÜK EĞİTİMİ(FAKÜLTE)',
 15: 'ANTRENÖRLÜK EĞİTİMİ(YÜKSEKOKUL)',
 16: 'ANTROPOLOJİ',
 17: 'ARAP DİLİ VE EDEBİYATI',
 18: 'ARAPÇA MÜTERCİM VE TERCÜMANLIK(FAKÜLTE)',
 19: 'ARAPÇA MÜTERCİM VE TERCÜMANLIK(YÜKSEKOKUL)',
 20: 'ARAPÇA ÖĞRETMENLİĞİ',
 21: 'ARKEOLOJİ',
 22: 'ARKEOLOJİ VE SANAT TARİHİ',
 23: 'ARNAVUT DİLİ VE EDEBİYATI',
 24: 'ASTRONOMİ VE UZAY BİLİMLERİ',
 25: 'AYAKKABI TASARIMI VE ÜRETİMİ',
 26: 'AZERBAYCAN TÜRKÇESİ VE EDEBİYA

In [79]:
def clean_major_name(major):
    if pd.isna(major):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    major = major.lower().strip()

    # Remove any digits
    major = re.sub(r"\d+", "", major)

    # Remove any parentheses and their contents
    major = re.sub(r"\([^)]*\)", "", major)

    # Remove any remaining punctuation and extra whitespace
    major = re.sub(r"[^\w\s]", "", major)
    major = " ".join(major.split())

    # Use fuzzy matching to find the closest match
    majors_list = list(majors.values())
    closest_match, score = process.extractOne(major, majors_list)

    if score >= 80:
        for key, value in majors.items():
            if value == closest_match:
                return key
    else:
        return major

In [80]:
train_data["bölüm"] = train_data["bölüm"].apply(clean_major_name)

In [81]:
train_data["bölüm"]

0                          44
1                         104
2                          34
3                          44
4                         365
                 ...         
65120                     291
65121    turizm işletmeciliği
65122                     310
65123                     345
65124                       1
Name: bölüm, Length: 65125, dtype: object

In [82]:
train_data["bölüm"] = pd.to_numeric(train_data["bölüm"], errors="coerce")
train_data["bölüm"].fillna(train_data["bölüm"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3012943815.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["bölüm"].fillna(train_data["bölüm"].mode()[0], inplace=True)


In [83]:
train_data["bölüm"].value_counts(dropna=False)

bölüm
230.0    13027
111.0     3522
44.0      2770
194.0     2489
104.0     2467
         ...  
446.0        1
122.0        1
174.0        1
428.0        1
298.0        1
Name: count, Length: 338, dtype: int64

In [84]:
train_data["bölüm"].unique()

array([ 44., 104.,  34., 365.,  30., 194., 230., 363., 111., 280., 323.,
       199.,  58., 117., 155.,  46.,   1., 269., 350., 432., 454.,  99.,
       274., 435., 359., 125., 285.,  36., 201., 163., 279.,   0., 328.,
       179., 176., 100., 384.,  29., 267.,  47., 403., 213., 434., 271.,
       295.,   7.,   9., 431., 309., 413., 152., 189., 302., 272., 203.,
       219., 289., 367., 366., 347., 429., 247., 330., 245., 414., 357.,
       106., 292., 165., 231., 371., 149., 427.,  73., 197.,  12., 120.,
       109., 275., 146., 205., 168., 345., 198., 316., 157., 291., 310.,
        49., 253., 242., 159., 404., 249., 162., 342., 353., 461., 234.,
        77., 286.,  43., 258., 447.,  16., 248., 326., 134., 317., 217.,
       307., 207., 419., 362., 327.,  91., 300.,   3., 140., 278., 329.,
       294., 405.,  85.,  22., 251., 206.,  83., 151., 313.,  13., 128.,
       282., 284., 358., 211., 277.,  68.,  80., 184., 221., 348., 312.,
       136.,  17.,   5., 349.,  42.,  66., 423.,  2

In [85]:
train_data["universite_kacinci_sinif"].unique()

array(['3', '1', 'nan', '2', '4', 'Hazırlık', 'Mezun', 'hazırlık',
       'Yüksek Lisans', '5', '6', 'Tez', '0'], dtype=object)

In [86]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [87]:
university_grade_map = {
    "nan": 0,
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "Mezun": 7,
    "Yüksek Lisans": 8,
    "Tez": 9,
    "hazırlık": 10,
}

In [88]:
train_data["universite_kacinci_sinif"] = pd.to_numeric(
    train_data["universite_kacinci_sinif"].map(university_grade_map),
    errors="coerce",
)
train_data["universite_kacinci_sinif"] = train_data["universite_kacinci_sinif"].fillna(
    0
)

In [89]:
train_data["universite_kacinci_sinif"].unique()

array([ 3.,  1.,  0.,  2.,  4.,  7., 10.,  8.,  5.,  6.,  9.])

In [90]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [91]:
train_data["universite_kacinci_sinif"].value_counts(dropna=False)

universite_kacinci_sinif
2.0     21773
3.0     16956
4.0     13113
1.0     10260
0.0      1652
5.0       961
7.0       178
6.0       177
10.0       33
8.0        15
9.0         7
Name: count, dtype: int64

In [92]:
train_data["universite_not_ortalamasi"].unique()

array(['3.50-3', '3.00-2.50', 'nan', '2.50 ve altı', '3.00 - 4.00',
       '3.50 - 4.00', '3.00 - 3.50', '2.50 -3.00', 'Ortalama bulunmuyor',
       'ORTALAMA BULUNMUYOR', '4-3.5', '2.50 - 3.00', '2.00 - 2.50',
       '1.00 - 2.50', 'Not ortalaması yok', '4.0-3.5', '3.00 - 3.49',
       '2.50 - 2.99', '1.80 - 2.49', 'Hazırlığım', '0 - 1.79'],
      dtype=object)

In [93]:
train_data["universite_not_ortalamasi"].value_counts()

universite_not_ortalamasi
3.00-2.50              10758
2.50 ve altı           10554
3.00 - 3.50             7913
3.50-3                  6179
2.50 - 3.00             5782
3.50 - 4.00             4462
nan                     2753
2.00 - 2.50             2519
2.50 - 2.99             2452
ORTALAMA BULUNMUYOR     2399
3.00 - 3.49             2382
2.50 -3.00              1734
3.00 - 4.00             1411
1.80 - 2.49             1166
Hazırlığım               668
Not ortalaması yok       586
4-3.5                    481
Ortalama bulunmuyor      413
1.00 - 2.50              327
0 - 1.79                 185
4.0-3.5                    1
Name: count, dtype: int64

In [94]:
train_data["universite_not_ortalamasi"].unique()

array(['3.50-3', '3.00-2.50', 'nan', '2.50 ve altı', '3.00 - 4.00',
       '3.50 - 4.00', '3.00 - 3.50', '2.50 -3.00', 'Ortalama bulunmuyor',
       'ORTALAMA BULUNMUYOR', '4-3.5', '2.50 - 3.00', '2.00 - 2.50',
       '1.00 - 2.50', 'Not ortalaması yok', '4.0-3.5', '3.00 - 3.49',
       '2.50 - 2.99', '1.80 - 2.49', 'Hazırlığım', '0 - 1.79'],
      dtype=object)

In [95]:
university_score_map = {
    "nan": "nan",
    "ORTALAMA BULUNMUYOR": "nan",
    "Hazırliğım": "nan",
    "Not Ortalaması yok": "nan",
    "Ortalama bulunmuyor": "nan",
    "2.50 ve altı": 1,
    "2.00 - 2.50": 1,
    "1.00 - 2.50": 1,
    "1.80 - 2.49": 1,
    "1 - 1.79": 1,
    "2.50 - 3.00": 2,
    "2.50 - 2.99": 2,
    "3.00-2.50": 2,
    "2.50 - 3.00": 2,
    "3.50-3": 3,
    "3.50 - 4.00": 3,
    "3.00 - 3.50": 3,
    "3.00 - 3.49": 3,
    "3.00 - 4.00": 3,
    "4-3.5": 4,
}

In [96]:
train_data["universite_not_ortalamasi"] = pd.to_numeric(
    train_data["universite_not_ortalamasi"].map(university_score_map),
    errors="coerce",
)

In [97]:
train_data["universite_not_ortalamasi"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3217699816.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["universite_not_ortalamasi"].fillna(0, inplace=True)


In [98]:
train_data["universite_not_ortalamasi"].unique()

array([3., 2., 0., 1., 4.])

In [99]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [100]:
train_data["universite_not_ortalamasi"].value_counts(dropna=False)

universite_not_ortalamasi
3.0    22347
2.0    18992
1.0    14566
0.0     8739
4.0      481
Name: count, dtype: int64

In [101]:
train_data["ingilizce_biliyor_musunuz?"].value_counts()

ingilizce_biliyor_musunuz?
Evet     48872
Hayır     9799
nan       6454
Name: count, dtype: int64

In [102]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,nan,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,nan,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,nan,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,nan,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,nan,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,Hayır,nan,65123,2000.0,11.0


In [103]:
do_you_know_english_map = {"Evet": 1, "Hayır": 0, "nan": "nan"}

In [104]:
train_data["ingilizce_biliyor_musunuz?"] = pd.to_numeric(
    train_data["ingilizce_biliyor_musunuz?"].map(do_you_know_english_map),
    errors="coerce",
)

In [105]:
train_data["ingilizce_biliyor_musunuz?"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/1022056692.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["ingilizce_biliyor_musunuz?"].fillna(0, inplace=True)


In [106]:
train_data["ingilizce_biliyor_musunuz?"].unique()

array([0., 1.])

In [107]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,nan,65123,2000.0,11.0


In [108]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,nan,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,nan,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,nan,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,nan,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,nan,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,nan,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,nan,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,nan,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,nan,65123,2000.0,11.0


In [109]:
def english_level_cleaner(df: pd.DataFrame):
    df.loc[
        df["ingilizce_biliyor_musunuz?"].isin(["nan", "0", 0]), "ingilizce_seviyeniz?"
    ] = 0

    # Handle cases where "ingilizce_biliyor_musunuz?" is 1
    df.loc[
        (df["ingilizce_biliyor_musunuz?"] == 1) & (df["ingilizce_seviyeniz?"] == "nan"),
        "ingilizce_seviyeniz?",
    ] = 0

    # Map English level strings to numerical values
    level_map = {"Başlangıç": 1, "Orta": 2, "İleri": 3}
    df["ingilizce_seviyeniz?"] = (
        df["ingilizce_seviyeniz?"].map(level_map).fillna(df["ingilizce_seviyeniz?"])
    )

    return df

In [110]:
train_data = english_level_cleaner(train_data)

In [111]:
train_data["ingilizce_seviyeniz?"].unique()

array([0, 3.0, 2.0, '0', 1.0], dtype=object)

In [112]:
train_data["ingilizce_seviyeniz?"] = train_data["ingilizce_seviyeniz?"].fillna(0)

In [113]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0,65123,2000.0,11.0


In [114]:
train_data.columns

Index(['basvuru_yili', 'degerlendirme_puani', 'cinsiyet', 'dogum_yeri',
       'ikametgah_sehri', 'universite_adi', 'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu?', 'bölüm',
       'universite_kacinci_sinif', 'universite_not_ortalamasi', 'lise_adi',
       'lise_sehir', 'lise_mezuniyet_notu', 'anne_egitim_durumu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id',
       'dogum_yili', 'dogum_ayi'],
      dtype='object')

In [115]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0,65123,2000.0,11.0


In [116]:
train_data["ingilizce_seviyeniz?"] = train_data["ingilizce_seviyeniz?"].astype("float")

In [117]:
train_data["anne_egitim_durumu"].value_counts().to_dict()

{'İlkokul': 10351,
 'Lise': 8467,
 'İlkokul Mezunu': 8030,
 'İLKOKUL MEZUNU': 7770,
 'Üniversite': 5502,
 'Eğitimi yok': 3605,
 'LİSE': 3582,
 'Ortaokul': 3126,
 'EĞİTİM YOK': 2600,
 'ÜNİVERSİTE': 2435,
 'Eğitim Yok': 2146,
 'Ortaokul Mezunu': 2119,
 'ORTAOKUL MEZUNU': 1788,
 'Lise Mezunu': 1346,
 'Üniversite Mezunu': 1075,
 'nan': 406,
 'Yüksek Lisans': 274,
 'YÜKSEK LİSANS': 153,
 'Yüksek Lisans / Doktora': 123,
 'Yüksek Lisans / Doktara': 85,
 'Doktora': 80,
 'DOKTORA': 62}

In [118]:
education = {
    "İlkokul": 1,
    "Lise": 3,
    "İLKOKUL MEZUNU": 1,
    "İlkokul Mezunu": 1,
    "Üniversite": 4,
    "LİSE": 3,
    "Eğitimi yok": 1,
    "Ortaokul": 2,
    "ÜNİVERSİTE": 4,
    "EĞİTİM YOK": 1,
    "Ortaokul Mezunu": 2,
    "Eğitim Yok": 1,
    "ORTAOKUL MEZUNU": 2,
    "Lise Mezunu": 3,
    "Üniversite Mezunu": 4,
    "nan": 0,
    "Yüksek Lisans": 5,
    "YÜKSEK LİSANS": 5,
    "Yüksek Lisans / Doktora": 6,
    "Yüksek Lisans / Doktara": 6,
    "Doktora": 6,
    "DOKTORA": 6,
}

In [119]:
train_data["anne_egitim_durumu"] = train_data["anne_egitim_durumu"].map(education)

In [120]:
train_data["anne_egitim_durumu"]

0        4
1        4
2        1
3        1
4        1
        ..
65120    1
65121    3
65122    3
65123    1
65124    1
Name: anne_egitim_durumu, Length: 65125, dtype: int64

In [121]:
train_data["anne_calisma_durumu"].value_counts(dropna=False)

anne_calisma_durumu
Hayır     39371
nan       11896
Evet      10504
Emekli     3354
Name: count, dtype: int64

In [122]:
train_data["baba_egitim_durumu"]

0                     Üniversite
1        Yüksek Lisans / Doktora
2                 İlkokul Mezunu
3                     Üniversite
4                     Üniversite
                  ...           
65120                   Ortaokul
65121                       Lise
65122                       Lise
65123                       Lise
65124                    İlkokul
Name: baba_egitim_durumu, Length: 65125, dtype: object

In [123]:
train_data["baba_egitim_durumu"] = train_data["baba_egitim_durumu"].map(education)
train_data["baba_egitim_durumu"] = train_data["baba_egitim_durumu"].fillna(0)

In [124]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [125]:
train_data["baba_egitim_durumu"].value_counts(dropna=False)

baba_egitim_durumu
1.0    21928
3.0    15895
4.0    15138
2.0     9180
0.0     1118
5.0      963
6.0      903
Name: count, dtype: int64

In [126]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [127]:
highschools = pd.read_csv(
    "./datathon-dataset/processed-data/highschools_of_istanbul.csv"
)
best_highschools = highschools.iloc[0:500, 0].str.split("/", expand=True)[2]
best_highschools.dropna(inplace=True)
best_highschools = best_highschools.str.strip()
best_highschools = best_highschools.to_dict()
best_highschools

{1: 'Galatasaray Üniversitesi Galatasaray Lisesi',
 2: 'İstanbul Erkek Lisesi',
 3: 'Kabataş Erkek Lisesi',
 4: 'Kabataş Erkek Lisesi',
 5: 'İstanbul Atatürk Fen Lisesi',
 6: 'Cağaloğlu Anadolu Lisesi',
 7: 'Çapa Fen Lisesi',
 8: 'Hüseyin Avni Sözen Anadolu Lisesi',
 9: 'Hüseyin Avni Sözen Anadolu Lisesi',
 10: 'Baykar Milli Teknoloji Mesleki Ve Teknik Anadolu Lisesi',
 11: 'Beşiktaş Sakıp Sabancı Anadolu Lisesi',
 12: 'Kartal Anadolu İmam Hatip Lisesi',
 13: 'Kadıköy Anadolu Lisesi',
 14: 'Haydarpaşa Lisesi',
 15: 'Baykar Milli Teknoloji Mesleki Ve Teknik Anadolu Lisesi',
 16: 'Vefa Lisesi',
 17: 'Haydarpaşa Lisesi',
 18: 'Yaşar Acar Fen Lisesi',
 19: 'Teknopark İstanbul Mesleki Ve Teknik Anadolu Lisesi',
 20: 'Burak Bora Anadolu Lisesi',
 21: 'Beşiktaş Anadolu Lisesi',
 22: 'Burak Bora Anadolu Lisesi',
 23: 'Validebağ Fen Lisesi',
 24: 'Kadıköy Anadolu İmam Hatip Lisesi',
 25: 'Pertevniyal Lisesi',
 26: 'Şehit Münir Alkan Fen Lisesi',
 27: 'İstanbul Ticaret Odası Marmara Anadolu İmam

In [128]:
threshold = 90


def is_similar_to_highschool(row):
    for key in best_highschools:
        if fuzz.partial_ratio(str(row), best_highschools[key]) > threshold:
            return True
    return False

In [129]:
train_data["lise_adi"] = train_data["lise_adi"].apply(
    lambda x: is_similar_to_highschool(x)
)

In [130]:
train_data["lise_adi"].value_counts()

lise_adi
False    58119
True      7006
Name: count, dtype: int64

In [131]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [132]:
train_data["lise_adi"].value_counts(dropna=False)

lise_adi
False    58119
True      7006
Name: count, dtype: int64

In [133]:
train_data["lise_mezuniyet_notu"].value_counts()

lise_mezuniyet_notu
75 - 100              22433
84-70                  9141
100-85                 6967
nan                    5917
4.00-3.50              3689
50 - 75                3115
3.00 - 4.00            3101
3.50-3.00              2759
3.50-3                 2330
69-55                  2162
3.00-2.50              2153
50 - 74                1007
2.50 ve altı            226
54-45                    88
25 - 50                  14
Not ortalaması yok       11
44-0                      6
0 - 25                    3
25 - 49                   2
0 - 24                    1
Name: count, dtype: int64

In [134]:
highschool_score = {
    "75 - 100": 4,
    "84-70": 4,
    "100-85": 4,
    "nan": pd.NA,
    "4.00-3.50": 4,
    "3.50-3.00": 4,
    "50 - 75": 3,
    "3.00 - 4.00": 4,
    "69-55": 3,
    "3.00-2.50": 3,
    "3.50-3": 4,
    "50 - 74": 3,
    "2.50 ve altı": 3,
    "54-45": 2,
    "Not ortalaması yok": pd.NA,
    "25 - 50": 2,
    "44-0": pd.NA,
    "25 - 49": 2,
    "0 - 25": 1,
    "0 - 24": 1,
}

In [135]:
train_data["lise_mezuniyet_notu"] = train_data["lise_mezuniyet_notu"].map(
    highschool_score
)
train_data["lise_mezuniyet_notu"] = pd.to_numeric(
    train_data["lise_mezuniyet_notu"], errors="coerce"
)
train_data["lise_mezuniyet_notu"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/4036599635.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["lise_mezuniyet_notu"].fillna(0, inplace=True)


In [136]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [137]:
train_data["lise_mezuniyet_notu"].value_counts(dropna=False)

lise_mezuniyet_notu
4.0    50420
3.0     8663
0.0     5934
2.0      104
1.0        4
Name: count, dtype: int64

In [138]:
working_situation_map = {"Hayır": 0, "nan": 0, "Evet": 1, "Emekli": 2}

In [139]:
train_data["anne_calisma_durumu"] = train_data["anne_calisma_durumu"].map(
    working_situation_map
)
train_data["anne_calisma_durumu"] = train_data["anne_calisma_durumu"].fillna(0)

In [140]:
train_data["baba_calisma_durumu"] = train_data["baba_calisma_durumu"].map(
    working_situation_map
)
train_data["baba_calisma_durumu"] = train_data["baba_calisma_durumu"].fillna(0)

In [141]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [142]:
train_data["baba_calisma_durumu"].value_counts(dropna=False)

baba_calisma_durumu
1    28559
0    25104
2    11462
Name: count, dtype: int64

In [143]:
train_data["anne_calisma_durumu"].value_counts(dropna=False)

anne_calisma_durumu
0    51267
1    10504
2     3354
Name: count, dtype: int64

In [144]:
sector_map = {
    "0": 0,
    "-": 0,
    "nan": 0,
    "Kamu": 1,
    "KAMU": 1,
    "Özel Sektör": 2,
    "ÖZEL SEKTÖR": 2,
    "Diğer": 3,
    "DİĞER": 3,
}

In [145]:
train_data["anne_sektor"] = train_data["anne_sektor"].map(sector_map)
train_data["anne_sektor"] = train_data["anne_sektor"].fillna(0)

In [146]:
train_data["anne_sektor"].value_counts(dropna=False)

anne_sektor
0    52144
2     6042
1     5149
3     1790
Name: count, dtype: int64

In [147]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [148]:
train_data["baba_sektor"].value_counts()

baba_sektor
nan            20378
Özel Sektör    11293
Kamu            7457
Diğer           6717
0               6631
ÖZEL SEKTÖR     4320
-               3070
KAMU            2862
DİĞER           2397
Name: count, dtype: int64

In [149]:
train_data["baba_sektor"] = train_data["baba_sektor"].map(sector_map)
train_data["baba_sektor"] = train_data["baba_sektor"].fillna(0)

In [150]:
train_data["baba_sektor"].value_counts(dropna=False)

baba_sektor
0    30079
2    15613
1    10319
3     9114
Name: count, dtype: int64

In [151]:
train_data["kardes_sayisi"].value_counts(dropna=False)

kardes_sayisi
1.0                                             16186
2.0                                             13062
3.0                                              6734
1                                                6054
4.0                                              4611
2                                                3938
0.0                                              3701
5.0                                              3421
3                                                2087
0                                                1381
nan                                              1132
4                                                 998
5                                                 715
6.0                                               206
7.0                                               159
6                                                 135
10.0                                              127
7                                                 119
8.0           

In [152]:
a = train_data["kardes_sayisi"].copy()

In [153]:
train_data["kardes_sayisi"] = pd.to_numeric(
    train_data["kardes_sayisi"], errors="coerce"
)
train_data["kardes_sayisi"] = train_data["kardes_sayisi"].fillna(0)

In [154]:
train_data["kardes_sayisi"].value_counts(dropna=False)

kardes_sayisi
1.0     22240
2.0     17000
3.0      8821
0.0      6215
4.0      5609
5.0      4136
6.0       341
7.0       278
8.0       173
10.0      169
9.0       122
11.0       12
12.0        5
13.0        2
18.0        1
14.0        1
Name: count, dtype: int64

In [155]:
train_data["lise_adi"] = train_data["lise_adi"].apply(lambda x: 1 if x == True else 0)

In [156]:
train_data.rename(
    columns={"lise_adi": "en_iyi_500_lise_içerisinde_mi_eğitim_gördü"}, inplace=True
)

In [157]:
first_part = train_data.copy()

In [158]:
train_data = first_part.copy()

In [159]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1.0,Evet,Evet,Hayır,Evet,0.0,0.0,0,1994.0,6.0
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,1.0,Hayır,Hayır,Hayır,Evet,0.0,0.0,1,1993.0,11.0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,6.0,Evet,Hayır,Hayır,Hayır,0.0,0.0,2,1986.0,1.0
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,10.0,Evet,Hayır,Hayır,Evet,0.0,0.0,3,1991.0,4.0
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,1.0,Hayır,Hayır,Hayır,Hayır,0.0,0.0,4,2000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65120,2003.0,6.0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,4.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65121,2002.0,4.0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,3.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65122,1999.0,10.0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,2.0,Hayır,Hayır,Hayır,nan,0.0,0.0,65123,2000.0,11.0


In [160]:
train_data["lise_sehir"] = train_data["lise_sehir"].apply(clean_city_name)

In [161]:
train_data["lise_sehir"].value_counts(dropna=False)

lise_sehir
34      13345
6        5167
35       3449
<NA>     2900
16       2360
        ...  
29         67
75         50
62         47
79          2
40          2
Name: count, Length: 82, dtype: int64

In [162]:
train_data["lise_sehir"].fillna(train_data["lise_sehir"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3314362712.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["lise_sehir"].fillna(train_data["lise_sehir"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16218/3314362712.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [163]:
club_map = {"Hayır": 0, "nan": 0, "Evet": 1}

In [164]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"].value_counts(
    dropna=False
)

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?
Hayır    33710
Evet     29586
nan       1829
Name: count, dtype: int64

In [165]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"]

0         Evet
1        Hayır
2         Evet
3         Evet
4        Hayır
         ...  
65120    Hayır
65121    Hayır
65122    Hayır
65123    Hayır
65124    Hayır
Name: girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?, Length: 65125, dtype: object

In [166]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"] = train_data[
    "girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"
].map(club_map)

train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"] = pd.to_numeric(
    train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"], errors="coerce"
)
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"] = train_data[
    "girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"
].fillna(0)

In [167]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].value_counts(
    dropna=False
)

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz
0    35539
1    29586
Name: count, dtype: int64

In [168]:
do_you_do_sports = club_map.copy()

In [169]:
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = train_data[
    "profesyonel_bir_spor_daliyla_mesgul_musunuz?"
].map(do_you_do_sports)

train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = pd.to_numeric(
    train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"], errors="coerce"
)
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = train_data[
    "profesyonel_bir_spor_daliyla_mesgul_musunuz?"
].fillna(0)

In [170]:
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"].value_counts(dropna=False)

profesyonel_bir_spor_daliyla_mesgul_musunuz?
0    43358
1    21767
Name: count, dtype: int64

In [171]:
are_you_in_a_stk_map = do_you_do_sports.copy()

In [172]:
train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = train_data[
    "aktif_olarak_bir_stk_üyesi_misiniz?"
].map(do_you_do_sports)

train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = pd.to_numeric(
    train_data["aktif_olarak_bir_stk_üyesi_misiniz?"], errors="coerce"
)
train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = train_data[
    "aktif_olarak_bir_stk_üyesi_misiniz?"
].fillna(0)

In [173]:
are_you_a_stk_member_map = are_you_in_a_stk_map.copy()

In [174]:
train_data["stk_projesine_katildiniz_mi?"] = train_data[
    "stk_projesine_katildiniz_mi?"
].map(do_you_do_sports)

train_data["stk_projesine_katildiniz_mi?"] = pd.to_numeric(
    train_data["stk_projesine_katildiniz_mi?"], errors="coerce"
)
train_data["stk_projesine_katildiniz_mi?"] = train_data[
    "stk_projesine_katildiniz_mi?"
].fillna(0)

In [175]:
train_data

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz
0,2014,52.0,0,6,6,23.0,1,100,1.0,44.0,...,1,1,0,1,0.0,0.0,0,1994.0,6.0,1
1,2014,30.0,0,34,34,23.0,1,0,0.0,104.0,...,0,0,0,1,0.0,0.0,1,1993.0,11.0,0
2,2014,18.0,0,55,34,0.0,1,0,0.0,34.0,...,1,0,0,0,0.0,0.0,2,1986.0,1.0,1
3,2014,40.0,0,21,34,112.0,1,100,1.0,44.0,...,1,0,0,1,0.0,0.0,3,1991.0,4.0,1
4,2014,24.0,0,6,6,169.0,1,100,1.0,365.0,...,0,0,0,0,0.0,0.0,4,2000.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,0,34,34,107.0,1,100,1.0,291.0,...,0,0,0,0,0.0,0.0,65120,2003.0,6.0,0
65121,2022,0.0,0,31,48,176.0,0,0,0.0,230.0,...,0,0,0,0,0.0,0.0,65121,2002.0,4.0,0
65122,2022,0.0,0,33,33,76.0,0,0,0.0,310.0,...,0,0,0,0,0.0,0.0,65122,1999.0,10.0,0
65123,2022,0.0,0,55,55,146.0,0,0,0.0,345.0,...,0,0,0,0,0.0,0.0,65123,2000.0,11.0,0


In [176]:
train_data.to_csv("./datathon-dataset/processed-data/train_data_cleaned.csv")